In [1]:
import pandas as pd
from xgboost import XGBClassifier
from collections import defaultdict
import pickle
import numpy as np

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from xlearn import FMModel,LRModel
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, \
balanced_accuracy_score, roc_auc_score, auc, precision_recall_curve, \
roc_curve
from sklearn.metrics import confusion_matrix

In [3]:
import os
os.chdir("D:/functional-prediction/")

In [4]:
df_variant = pd.read_csv("output/variant_feat_clean.csv")
df_apf = pd.read_csv("output/apf_feat_clean.csv")

In [5]:
df_feat = pd.concat([df_variant, df_apf], axis=0)

In [6]:
# 去重
df_feat["function"] = ["neutral" if "normal" in x or x == "N" else "deleterious" for x in df_feat["function"].values]
df_feat["chr"] = [str(x) for x in df_feat["chr"].values]
df_feat = df_feat.drop_duplicates(["gene", "variant_start", "reference_allele", "variant_allele", "function"])
_df = df_feat.groupby(
    ["gene", "variant_start", "reference_allele", "variant_allele"]
)["function"].count().reset_index(name='count').sort_values(by=["count"], ascending=False)
_df = _df[_df["count"] == 2]
_df["function_new"] = ["neutral"] * len(_df)
_df.pop('count')

df_feat = pd.merge(df_feat, _df, how='left', on=["gene", "variant_start", "reference_allele", "variant_allele"])
df_dup = df_feat[df_feat["function_new"].notnull()]
df_dup["function"] = ["neutral"] * len(df_dup)
df_dup = df_dup.drop_duplicates(["gene", "variant_start", "reference_allele", "variant_allele", "function"])

df_not_dup = df_feat[~df_feat["function_new"].notnull()]
df_feat = pd.concat([df_dup, df_not_dup], axis=0)
df_feat.pop("function_new")
print(len(df_feat))

535


d:\anaconda\envs\drkg\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
df_feat.groupby(["function"])["gene"].count().reset_index(name='count')

,function,count
0,deleterious,385
1,neutral,150


In [8]:
_df = df_feat.groupby(["gene"])["function"].count().reset_index(name='count')
", ".join(list(_df["gene"].values))

'ABCC2, CYP17A1, CYP1A2, CYP1B1, CYP21A2, CYP2A13, CYP2A6, CYP2B6, CYP2C19, CYP2C9, CYP2D6, DPYD, FMO3, G6PD, GSTP1, HNMT, NAT1, NAT2, NR1I2, NUDT15, PNMT, RYR1, SLC22A12, SLC2A9, SLC47A1, SLCO1B1, TPMT, UGT1A1, UGT1A10, UGT1A4, UGT1A8, UGT1A9'

In [9]:
seed1 = 14398733
seed2 = 1396232

In [10]:
# 5 fold split
# good seeds: (43987334， 3962321)
function_dict = {}
fold = 6 # 1 test set + 5 train-validation set 

for function, content in df_feat.groupby(["function"]):
    content = content.sample(frac=1, random_state=seed1).reset_index(drop=True)

    fold_size = int(len(content) / fold) + 1
    
    content_list = []
    for i in range(fold):
        content_part = content.iloc[i * fold_size: (i + 1) * fold_size]
        content_part.index = range(len(content_part))
        content_list.append(content_part)
    
    function_dict[function] = content_list

# get test data

In [11]:
test_list = []
test_partition = 5

for key, value in function_dict.items():
    test_list.append(value[test_partition])
    
df_test = pd.concat(test_list, axis=0)
df_test_ = df_test.copy(deep=True)

test_label = df_test["function"].values
test_variant_start = df_test["variant_start"].values
test_apf = df_test["APF_score"].values
for col in ["gene", "haplotype_name", "chr", "variant_start", 
                "reference_allele", "variant_allele", "function", 
                "variant", "type"]:
    df_test_.pop(col)


# get train data

In [12]:
# total set training

part_dict = defaultdict(list)
for key, value in function_dict.items():    
    for j in range(fold):
        if j == test_partition:
            continue
        part_dict[key].append(value[j])
        
# balancing traning set
for key in part_dict.keys():
    part_dict[key] = pd.concat(part_dict[key], axis=0)

# upsampling with 2 * max_len
max_len = max([len(x) for x in part_dict.values()])
for key in part_dict.keys():
    part_dict[key] = part_dict[key].sample(n=max_len*2, replace=True, random_state=seed2)

df_train = pd.concat(list(part_dict.values()), axis=0)

train_label = df_train["function"].values

df_train_ = df_train.copy(deep=True)

for col in ["gene", "haplotype_name", "chr", "variant_start", 
                "reference_allele", "variant_allele", "function", 
                "variant", "type"]:
    df_train.pop(col)
    
# train_data = df_train.values


In [13]:
imputer_dict = {}
normalizer_dict = {}
for col in df_train.columns:
    if "_variant" in col or "_gained" in col or "_lost" in col:
        continue
    imp = SimpleImputer(missing_values=np.nan, strategy='mean').fit(df_train[col].values.reshape(-1, 1))
    imputer_dict[col] = imp
    df_train[col] = imp.transform(df_train[col].values.reshape(-1, 1))
    
    normalizer = StandardScaler().fit(df_train[col].values.reshape(-1, 1))
    normalizer_dict[col] = normalizer
    df_train[col] = normalizer.transform(df_train[col].values.reshape(-1, 1))

with open("model/imputer_dict.pkl", "wb") as f:
    pickle.dump(imputer_dict, f)
    
with open("model/normalizer_dict.pkl", "wb") as f:
    pickle.dump(normalizer_dict, f)

In [14]:
with open("model/imputer_dict.pkl", "rb") as f:
    imputer_dict = pickle.load(f)
    
with open("model/normalizer_dict.pkl", "rb") as f:
    normalizer_dict = pickle.load(f)


In [15]:
for key in imputer_dict.keys():
    df_test_[key] = imputer_dict[key].transform(df_test_[key].values.reshape(-1, 1))
    df_test_[key] = normalizer_dict[key].transform(df_test_[key].values.reshape(-1, 1))

In [33]:
# 补充关键数据项数据
df_train_key = df_train.copy(deep=True)
col_list = [
        'DEOGEN2_score', 'M-CAP_score', 'MPC_score', 'MutationAssessor_score',
        'LRT_score', 'FATHMM_score', 'PROVEAN_score',
        'Polyphen2_HVAR_score', 'integrated_fitCons_score', 'VEST4_score',
        'SIFT4G_score', 'LoFtool', 'GenoCanyon_score', 'CADD_raw', 'APF_score'
    ]
for col in df_train_key.columns:
    if "_variant" in col or "_gained" in col or "_lost" in col:
        col_list.append(col)
        continue
        
    if col not in [
        'DEOGEN2_score', 'M-CAP_score', 'MPC_score', 'MutationAssessor_score',
        'LRT_score', 'FATHMM_score', 'PROVEAN_score',
        'Polyphen2_HVAR_score', 'integrated_fitCons_score', 'VEST4_score',
        'SIFT4G_score', 'LoFtool', 'GenoCanyon_score', 'CADD_raw', 'APF_score'
    ]:
        df_train_key[col] = [np.nan] * len(df_train_key)
        df_train_key[col] = imputer_dict[col].transform(df_train_key[col].values.reshape(-1, 1))
        df_train_key[col] = normalizer_dict[col].transform(df_train_key[col].values.reshape(-1, 1))
        
        
# df_train_all = pd.concat([df_train, df_train_key], axis=0)

df_train_key = df_train.copy(deep=True)

df_train = df_train[col_list]
df_test_ = df_test_[col_list]

for col in df_train_key.columns:
    if "_variant" in col or "_gained" in col or "_lost" in col:
        continue
        
    if col not in [
        'DEOGEN2_score', 'M-CAP_score', 'MPC_score', 'MutationAssessor_score',
        'LRT_score', 'FATHMM_score', 'PROVEAN_score',
        'Polyphen2_HVAR_score', 'integrated_fitCons_score', 'VEST4_score',
        'SIFT4G_score', 'LoFtool', 'GenoCanyon_score', 'CADD_raw', 'APF_score'
    ]:
        df_train_key[col] = [np.nan] * len(df_train_key)
        df_train_key[col] = imputer_dict[col].transform(df_train_key[col].values.reshape(-1, 1))
        df_train_key[col] = normalizer_dict[col].transform(df_train_key[col].values.reshape(-1, 1))
        
        
# df_train_all = pd.concat([df_train_all, df_train_key], axis=0)

df_train_all = df_train

In [34]:
def get_performance(test_label, result, y_pred=None):
    result = ["neutral" if x == "neutrual" else x for x in result]
    test_label = ["neutral" if x == "neutrual" else x for x in test_label]
    test_label_ = [0 if x == "neutral" else 1 for x in test_label]
    
    if y_pred is not None:
        roc_auc = roc_auc_score(test_label_, y_pred)
        print("roc_auc:", round(roc_auc, 4))

        precision, recall, thresholds = precision_recall_curve(test_label_, y_pred)
        f1 = [(2 * precision[i] * recall[i]) / (precision[i] + recall[i]) for i in range(len(thresholds))]
        max_thres = max(zip(f1, thresholds), key=lambda x: x[0])
        print("best threshold: ", round(max_thres[1], 4))
        print("best f1: ", round(max_thres[0], 4))
        
        prc_auc = auc(recall, precision)
        print("prc_auc:", round(prc_auc, 4))
    
    _test_label = [0 if x == "neutral" else 1 for x in test_label]
    _result = [0 if x == "neutral" else 1 for x in result]
    tn, fp, fn, tp = confusion_matrix(test_label, result).ravel()
    specificity = tn / (tn + fp)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    
    print("precision:", round(precision, 4))
    print("sensitivity/recall:", round(recall, 4))
    print("f1:", round(f1, 4))
    print("specificity:", round(specificity, 4))

In [56]:
def get_auc(y_true, y_pred, pos_label=None):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred, pos_label=pos_label)
    # x: fpr 假阳率
    # y: tpr 召回率
    roc_auc = round(auc(fpr, tpr), 4)
    return fpr, tpr, roc_auc

In [35]:
test_data_fillna = df_test_.values
train_data_fillna = df_train_all.values


# --------------------------------------------- 

In [58]:
auc_dict = {}
auc_value_dict = {}

In [59]:
# xgboost model
xgb_model = XGBClassifier(
    learning_rate=0.2,
    subsample=0.6,
    max_depth=4,
    n_estimators=80,
    objective="binary:logistic"
)

xgb_model.fit(
    train_data_fillna, list(train_label),
    eval_metric='auc'
)

# result = list(xgb_model.predict(test_data_fillna))
y_pred = xgb_model.predict_proba(test_data_fillna)[:,0]
result = ["neutral" if x <= 0.5 else "deleterious" for x in y_pred]
xgb_model.result = result
xgb_model.result_proba = y_pred

get_performance(test_label, result, y_pred)

fpr, tpr, roc_auc = get_auc(test_label, y_pred, pos_label="deleterious")
auc_dict["xgboost_fpr"] = fpr
auc_dict["xgboost_tpr"] = tpr
auc_value_dict["xgboost"] = roc_auc

roc_auc: 0.9267
best threshold:  0.1962
best f1:  0.9421
prc_auc: 0.9742
precision: 0.6957
sensitivity/recall: 0.8
f1: 0.7442
specificity: 0.8833


d:\anaconda\envs\drkg\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [60]:
# lightgbm model
lgb_model = LGBMClassifier(
    max_depth=4, 
    learning_rate=0.2, 
    n_estimators=60,
    subsample=0.6, 
    reg_lambda=0.1
)

_train_label = [0 if x == "neutral" else 1 for x in train_label]
_test_label = [0 if x == "neutral" else 1 for x in test_label]

lgb_model.fit(
   train_data_fillna, _train_label,
)

y_pred = lgb_model.predict_proba(test_data_fillna)[:,1]
result = ["neutral" if x <= 0.5 else "deleterious" for x in y_pred]
lgb_model.result = result
lgb_model.result_proba = y_pred

get_performance(test_label, result, y_pred)

fpr, tpr, roc_auc = get_auc(test_label, y_pred, pos_label="deleterious")
auc_dict["lightgbm_fpr"] = fpr
auc_dict["lightgbm_tpr"] = tpr
auc_value_dict["lightgbm"] = roc_auc

roc_auc: 0.9058
best threshold:  0.1653
best f1:  0.9333
prc_auc: 0.9616
precision: 0.68
sensitivity/recall: 0.85
f1: 0.7556
specificity: 0.8667


In [38]:
# fm model
# 需要对空值进行处理

# fm_model = FMModel(
#     task='binary', 
#     init=0.1,
#     epoch=50, 
#     lr=0.12,
#     reg_lambda=0.01,
#     stop_window=2,
#     k=4,
# )

# _train_label = [0 if x == "neutral" else 1 for x in train_label]
# _test_label = [0 if x == "neutral" else 1 for x in test_label]

# fm_model.fit(
#    train_data_fillna, _train_label * 2,
#     is_lock_free=False
# )

# y_pred = fm_model.predict(test_data_fillna)
# result = ["neutral" if x <= 0.5 else "deleterious" for x in y_pred]
# fm_model.result = result
# fm_model.result_proba = y_pred

# get_performance(test_label, result, y_pred)

In [39]:
# lr model
# 需要对空值进行处理
# lr_model = LRModel(
#     task='binary', 
#     init=0.1,
#     epoch=50, 
#     lr=0.01,
#     reg_lambda=0.2,
#     stop_window=4,
# )

# _train_label = [0 if x == "neutral" else 1 for x in train_label]
# _test_label = [0 if x == "neutral" else 1 for x in test_label]

# lr_model.fit(
#    train_data_fillna, _train_label * 2,
#     is_lock_free=False
# )

# y_pred = lr_model.predict(test_data_fillna)
# result = ["neutral" if x <= 0.5 else "deleterious" for x in y_pred]
# lr_model.result = result
# lr_model.result_proba = y_pred

# get_performance(test_label, result, y_pred)

In [62]:
# rf model
rf_model = RandomForestClassifier(
    max_depth=6,
    random_state=206
)

rf_model.fit(
   train_data_fillna, _train_label,
)

y_pred = rf_model.predict_proba(test_data_fillna)[:,1]
result = ["neutral" if x <= 0.5 else "deleterious" for x in y_pred]
rf_model.result = result
rf_model.result_proba = y_pred

get_performance(test_label, result, y_pred)

fpr, tpr, roc_auc = get_auc(test_label, y_pred, pos_label="deleterious")
auc_dict["rf_fpr"] = fpr
auc_dict["rf_tpr"] = tpr
auc_value_dict["rf"] = roc_auc

roc_auc: 0.9333
best threshold:  0.2413
best f1:  0.9355
prc_auc: 0.9771
precision: 0.7619
sensitivity/recall: 0.8
f1: 0.7805
specificity: 0.9167


In [41]:
##### df_test_show = df_test[["gene", "haplotype_name", "chr", "variant_start", "reference_allele", "variant_allele", "function"]]
# df_test_show.index = range(len(df_test_show))
# df_test_show["function_prediction"] = result
# df_test_show["function"] = test_label
# df_test_show = df_test_show.sort_values(by=["gene", "function"])
# df_test_show.index = range(len(df_test_show))
# df_test_show = df_test_show.fillna("")

In [42]:
# def most_common(lst):
#     return max(lst, key=lst.count)

# hard_vote = [
#     most_common(x) for x in list(zip(
#         lr_model.result,
#         fm_model.result,
#         xgb_model.result,
#         lgb_model.result,
#         rf_model.result
#     ))
# ]

# get_performance(test_label, hard_vote)

In [43]:
# xgb, lgb, rf
def most_common(lst):
    return max(lst, key=lst.count)

hard_vote = [
    most_common(x) for x in list(zip(
        xgb_model.result,
        lgb_model.result,
        rf_model.result
    ))
]

get_performance(test_label, hard_vote)

precision: 0.7273
sensitivity/recall: 0.8
f1: 0.7619
specificity: 0.9


In [44]:
# soft_vote_proba = [sum(x) for x in
#     list(zip(
#         lr_model.result_proba,
#         fm_model.result_proba,
#         xgb_model.result_proba,
#         lgb_model.result_proba,
#         rf_model.result_proba
#     ))
# ]

# soft_vote = ["deleterious" if x > 1.63 else "neutral" for x in soft_vote_proba]

# get_performance(test_label, soft_vote, soft_vote_proba)

In [63]:
# xgb, lgb, rf
soft_vote_proba = [sum(x) for x in
    list(zip(
        xgb_model.result_proba,
        lgb_model.result_proba,
        rf_model.result_proba
    ))
]

soft_vote = ["deleterious" if x > 0.75 else "neutral" for x in soft_vote_proba]

get_performance(test_label, soft_vote, soft_vote_proba)

fpr, tpr, roc_auc = get_auc(test_label, soft_vote_proba, pos_label="deleterious")
auc_dict["soft_vote_fpr"] = fpr
auc_dict["soft_vote_tpr"] = tpr
auc_value_dict["soft_vote"] = roc_auc

roc_auc: 0.935
best threshold:  0.7183
best f1:  0.9421
prc_auc: 0.9778
precision: 0.8
sensitivity/recall: 0.8
f1: 0.8
specificity: 0.9333


In [88]:
# APF
y_pred = test_apf
result = ["neutral" if x <= 0.5 else "deleterious" for x in y_pred]

get_performance(test_label, result, y_pred)

fpr, tpr, roc_auc = get_auc(test_label, y_pred, pos_label="deleterious")
auc_dict["apf_fpr"] = fpr
auc_dict["apf_tpr"] = tpr
auc_value_dict["apf"] = roc_auc

roc_auc: 0.8642
best threshold:  0.167
best f1:  0.9062
prc_auc: 0.9454
precision: 0.4595
sensitivity/recall: 0.85
f1: 0.5965
specificity: 0.6667


In [89]:
max_len = max([len(x) for x in auc_dict.values()])

for key, value in auc_dict.items():
    if len(value) < max_len:
        auc_dict[key] = list(value) + [np.nan] * (max_len - len(value))
        
pd.DataFrame(auc_dict).to_excel("output/auc.xlsx", index=False)

In [47]:
# save all model
# xlearn dump file has bug

from datetime import datetime

today_str = datetime.now().strftime("%m%d")

def save_model():
    with open("model/xgb_binary_{}.pkl".format(today_str), "wb") as f:
        pickle.dump(xgb_model, f)
    
    with open("model/lgb_binary_{}.pkl".format(today_str), "wb") as f:
        pickle.dump(lgb_model, f)
    
    with open("model/rf_binary_{}.pkl".format(today_str), "wb") as f:
        pickle.dump(rf_model, f)
        
    with open("model/col_list_binary_{}.pkl".format(today_str), "wb") as f:
        pickle.dump(col_list, f)
        
save_model()

# test missing feature

In [112]:
# 缺少特征时只使用随机森林作为分类器
# with open("model/imputer_dict.pkl", "rb") as f:
#     imputer_dict = pickle.load(f)

# with open("model/normalizer_dict.pkl", "rb") as f:
#     normalizer_dict = pickle.load(f)

# with open("model/rf_binary_0816.pkl", "rb") as f:
#     rf_model = pickle.load(f)
    
df_test_key = df_test_.copy(deep=True)

# # 将非必须列设为空，测试性能
# for col in df_test_key.columns:
#     if "_variant" in col or "_gained" in col or "_lost" in col:
#         continue
        
#     if col not in [
#         "integrated_fitCons_score", "MetaLR_score", "MPC_score",
#         "CADD_raw", "SIFT4G_score", "BayesDel_addAF_score", "LRT_score",
#         "MutationAssessor_score", "PROVEAN_score", "VEST4_score", "DEOGEN2_score",
#         "FATHMM_score", "LoFtool", "APF_score"
#     ]:
#         df_test_key[col] = [np.nan] * len(df_test_key)
        

# df_test_key = df_test_.copy(deep=True)

# # 将非必须列设为空，测试性能
# for col in df_test_key.columns:
#     if "_variant" in col or "_gained" in col or "_lost" in col:
#         continue
        
#     if col not in [
#         "MetaLR_score", "CADD_raw", "SIFT4G_score", "LRT_score",
#         "MutationAssessor_score", "PROVEAN_score", "VEST4_score", 
#         "LoFtool", "APF_score"
#     ]:
#         df_test_key[col] = [np.nan] * len(df_test_key)

In [113]:
for key in imputer_dict.keys():
    df_test_key[key] = imputer_dict[key].transform(df_test_key[key].values.reshape(-1, 1))
    df_test_key[key] = normalizer_dict[key].transform(df_test_key[key].values.reshape(-1, 1))

In [114]:
df_test_key["variant_start"] = test_variant_start
df_test_key["result"] = test_label
df_test_key["apf_raw"] = test_apf

df_test_apf = df_test_key[df_test_key["variant_start"].isin(df_apf["variant_start"].values)]
apf_test_label = df_test_apf["result"].values
apf_test_apf = df_test_apf["apf_raw"].values

df_test_key.pop("result")
df_test_apf.pop("result")
df_test_key.pop("variant_start")
df_test_apf.pop("variant_start")
df_test_key.pop("apf_raw")
df_test_apf.pop("apf_raw")
print()

In [115]:
xgb_pred = xgb_model.predict_proba(df_test_key.values)[:, 0]
lgb_pred = lgb_model.predict_proba(df_test_key.values)[:, 1]
rf_pred = rf_model.predict_proba(df_test_key.values)[:,1]

def most_common(lst):
    return max(lst, key=lst.count)

hard_vote = [
    most_common(x) for x in list(zip(
        ["neutral" if x <= 0.5 else "deleterious" for x in xgb_pred],
        ["neutral" if x <= 0.5 else "deleterious" for x in lgb_pred],
        ["neutral" if x <= 0.5 else "deleterious" for x in rf_pred]
    ))
]
print("our dataset:")
print("hard:")
get_performance(test_label, hard_vote)

soft_vote_proba = [sum(x) for x in
    list(zip(
        xgb_pred,
        lgb_pred,
        rf_pred
    ))
]

soft_vote = ["neutral" if x <= 1.6203 else "deleterious" for x in soft_vote_proba]
print("soft:")
get_performance(test_label, soft_vote, soft_vote_proba)

# APF
y_pred = apf_test_apf
result = ["neutral" if x <= 0.333 else "deleterious" for x in y_pred]
print("apf:")
get_performance(apf_test_label, result, y_pred)

our dataset:
hard:
precision: 0.6154
sensitivity/recall: 0.9231
f1: 0.7385
specificity: 0.7692
soft:
roc_auc: 0.9107
best threshold:  1.0498
best f1:  0.896
prc_auc: 0.967
precision: 0.5455
sensitivity/recall: 0.9231
f1: 0.6857
specificity: 0.6923
apf:
roc_auc: 0.6278
best threshold:  0.143
best f1:  0.7143
prc_auc: 0.6721
precision: 0.5294
sensitivity/recall: 0.6
f1: 0.5625
specificity: 0.5556


In [116]:
xgb_pred = xgb_model.predict_proba(df_test_apf.values)[:, 0]
lgb_pred = lgb_model.predict_proba(df_test_apf.values)[:, 1]
rf_pred = rf_model.predict_proba(df_test_apf.values)[:,1]

def most_common(lst):
    return max(lst, key=lst.count)

hard_vote = [
    most_common(x) for x in list(zip(
        ["neutral" if x <= 0.5 else "deleterious" for x in xgb_pred],
        ["neutral" if x <= 0.5 else "deleterious" for x in lgb_pred],
        ["neutral" if x <= 0.5 else "deleterious" for x in rf_pred]
    ))
]
print("apf dataset:")
print("hard:")
get_performance(apf_test_label, hard_vote)

soft_vote_proba = [sum(x) for x in
    list(zip(
        xgb_pred,
        lgb_pred,
        rf_pred
    ))
]

soft_vote = ["neutral" if x <= 0.75 else "deleterious" for x in soft_vote_proba]
print("soft:")
get_performance(apf_test_label, soft_vote, soft_vote_proba)

apf dataset:
hard:
precision: 0.5909
sensitivity/recall: 0.8667
f1: 0.7027
specificity: 0.5
soft:
roc_auc: 0.8148
best threshold:  0.5909
best f1:  0.8372
prc_auc: 0.8491
precision: 0.6923
sensitivity/recall: 0.6
f1: 0.6429
specificity: 0.7778
